<a href="https://colab.research.google.com/github/hjy530/CatsGreatAdventure/blob/main/prototype/RecommendationSystem/Hybrid_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**<프로토타입용 추천 시스템>**

- 하이브리드 기반 추천 시스템(Hybrid Recommender Systems)

(사용한) 머신러닝을 위한 파이썬 라이브러리
- Numpy(넘파이)
- Pandas(판다스)
- sklearn(사이킷런)
- surprise(서프라이즈)

* 두 가지 필터링 방법을 결합하여 하나의 추천 시스템을 개발한다.
  - 해당 유저와 비슷한 유저가 구매/조회한 아이템이면서, 그 아이템의 다른 추가 속성(ex. 유사한 장르/유사한 카테고리 등)에 있는 다른 아이템도 추가로 추천하는 방식

- [하이브리드 기반 추천 시스템]
  - 콘텐츠 기반 필터링과 협업 기반 필터링을 합친 추천 시스템
  - CF(Collaborative Filtiering) 점수에 콘텐츠 기반의 점수를 곱해서 정렬 후 사용

  - 하이브리드 모델 방식: 콘텐츠 기반 필터링에서 시대와 기법을 이용하여 유사한 명화를 추천해주는 방식과 협업 기반 필터링에서 평점 데이터를 이용해 개인이 평점을 남긴 평점 데이터를 기반으로 새롭게 모든 명화의 평점을 예측하여 높은 예측 평점 중 개인이 평점을 부여하지 않은 명화만을 추천하는 방식을 결합하여 만든다.

----------

- 필요한 라이브러리 import 

In [ ]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')

In [ ]:
from surprise import SVD, BaselineOnly, SVDpp, NMF, SlopeOne, CoClustering, Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise import dump

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

- 데이터 준비 및 전처리



In [ ]:
def convert_traintest_dataframe_forsurprise(training_dataframe, testing_dataframe):
    reader = Reader(rating_scale=(0, 5))
    trainset = Dataset.load_from_df(training_dataframe[['user_id', 'masterpiece_seq', 'grade_score']], reader)
    testset = Dataset.load_from_df(testing_dataframe[['user_id', 'masterpiece_seq', 'grade_score']], reader)
    trainset = trainset.construct_trainset(trainset.raw_ratings)
    testset = testset.construct_testset(testset.raw_ratings)
    return trainset, testset

In [ ]:
masterpiece = pd.read_csv('https://raw.githubusercontent.com/hjy530/WOOYE/main/prototype/csv/masterpiece.csv', encoding='utf-8')
grade = pd.read_csv('https://raw.githubusercontent.com/hjy530/WOOYE/main/prototype/csv/grade.csv', encoding='utf-8')

masterpiece_grade = pd.merge(masterpiece, grade, on="masterpiece_seq")
masterpiece_grade.drop(['masterpiece_picture', 'masterpiece_footnote'], axis=1, inplace=True)

traindf = masterpiece_grade
testdf = masterpiece_grade
trainset, testset = convert_traintest_dataframe_forsurprise(traindf, testdf)

In [ ]:
testdf.head()

,masterpiece_seq,masterpiece_title,masterpiece_artist,masterpiece_era,masterpiece_year,masterpiece_technique,masterpiece_material,grade_seq,user_id,grade_score
0,1,모나리자,레오나르도 다 빈치,르네상스,1503.0,인물화,유화,2,이소이,8
1,1,모나리자,레오나르도 다 빈치,르네상스,1503.0,인물화,유화,44,이루다,1
2,1,모나리자,레오나르도 다 빈치,르네상스,1503.0,인물화,유화,92,지사해,8
3,1,모나리자,레오나르도 다 빈치,르네상스,1503.0,인물화,유화,135,탁재영,4
4,1,모나리자,레오나르도 다 빈치,르네상스,1503.0,인물화,유화,209,김수연,6


- CF KNN 모델

In [ ]:
# basic collaborative filtering algorithm taking into account a baseline rating.
sim_options = {'name': 'cosine',
               'user_based': False  # compute  similarities between items
               }
knnbaseline_algo = KNNBaseline(sim_options=sim_options)

knnbaseline_algo.fit(trainset)
knnbaseline_predictions = knnbaseline_algo.test(testset)

file_name = 'KnnBaseline_model'
dump.dump(file_name, algo=knnbaseline_predictions)
# _, loaded_algo = dump.load(file_name)

accuracy.rmse(knnbaseline_predictions)
accuracy.mae(knnbaseline_predictions)
print("Done!")

Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 2.8936
MAE:  2.4975
Done!


- CF SVD 모델

In [ ]:
svdpp_algo = SVDpp()

svdpp_algo.fit(trainset)
svdpp_predictions = svdpp_algo.test(testset)

file_name = 'svd_model'
dump.dump(file_name, algo=svdpp_algo)
# _, loaded_algo = dump.load(file_name)

accuracy.rmse(svdpp_predictions)
accuracy.mae(svdpp_predictions)
print("Done!")

RMSE: 2.5069
MAE:  2.0019
Done!


- Masterpiece Similarity model

In [ ]:
era_to_idx = {
    '르네상스' : 1,
    '바로크' : 2,
    '로코코' : 3,
    '고전주의' : 4,
    '낭만주의' : 5,
    '사실주의' : 6,
    '인상주의' : 7,
    '표현주의' : 8,
    '초현실주의' : 9,
    '팝아트' : 10,
}

In [ ]:
idx_to_era = {
    1 : '르네상스',
    2 : '바로크',
    3 : '로코코',
    4 : '고전주의',
    5 : '낭만주의',
    6 : '사실주의',
    7 : '인상주의',
    8 : '표현주의',
    9 : '초현실주의',
    10 : '팝아트',
}

In [ ]:
masterpieces = masterpiece[['masterpiece_seq', 'masterpiece_title', 'masterpiece_artist', 'masterpiece_era', 'masterpiece_year', 'masterpiece_technique', 'masterpiece_material']]

# masterpieces['description_era'] = masterpieces['masterpiece_technique'] + 2*masterpieces['masterpiece_era']
masterpieces['description_era'] = masterpieces['masterpiece_era']
masterpieces['description_era'] = masterpieces['description_era'].fillna('')

In [ ]:
tf_new = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0)
tfidf_matrix_new = tf_new.fit_transform(masterpieces['description_era'])

In [ ]:
cosine_sim_new = linear_kernel(tfidf_matrix_new, tfidf_matrix_new)

In [ ]:
masterpieces = masterpieces.reset_index()
titles = masterpieces['masterpiece_title']
indices = pd.Series(masterpieces.index, index=masterpieces['masterpiece_title'])
indices.head(2)

masterpiece_title
모나리자    0
천지창조    1
dtype: int64

In [ ]:
def get_recommendations_new(title):
    idx = indices[title]
    if type(idx) != np.int64:
        if len(idx)>1:
            print("ALERT: Multiple values")
            idx = idx[0]
    sim_scores = list(enumerate(cosine_sim_new[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    # print(sim_scores)
    masterpiece_indices = [i[0] for i in sim_scores]
    # print(masterpiece_indices)
    return masterpieces['masterpiece_seq'].iloc[masterpiece_indices]

In [ ]:
get_recommendations_new('오감')

6      7
7      8
8      9
9     10
10    11
11    12
12    13
0      1
1      2
2      3
Name: masterpiece_seq, dtype: int64

- Popularity model

In [ ]:
masterpiece_grade.head(20)

,masterpiece_seq,masterpiece_title,masterpiece_artist,masterpiece_era,masterpiece_year,masterpiece_technique,masterpiece_material,grade_seq,user_id,grade_score
0,1,모나리자,레오나르도 다 빈치,르네상스,1503.0,인물화,유화,2,이소이,8
1,1,모나리자,레오나르도 다 빈치,르네상스,1503.0,인물화,유화,44,이루다,1
2,1,모나리자,레오나르도 다 빈치,르네상스,1503.0,인물화,유화,92,지사해,8
3,1,모나리자,레오나르도 다 빈치,르네상스,1503.0,인물화,유화,135,탁재영,4
4,1,모나리자,레오나르도 다 빈치,르네상스,1503.0,인물화,유화,209,김수연,6
5,1,모나리자,레오나르도 다 빈치,르네상스,1503.0,인물화,유화,223,남리을,7
6,1,모나리자,레오나르도 다 빈치,르네상스,1503.0,인물화,유화,342,신아연,7
7,1,모나리자,레오나르도 다 빈치,르네상스,1503.0,인물화,유화,385,서희수,4
8,1,모나리자,레오나르도 다 빈치,르네상스,1503.0,인물화,유화,394,김성연,2
9,1,모나리자,레오나르도 다 빈치,르네상스,1503.0,인물화,유화,444,최수현,1


In [ ]:
def era_based_popularity(era):
    mask = masterpiece_grade.masterpiece_era.apply(lambda x: era in x)
    filtered_masterpiece = masterpiece_grade[mask]
    filtered_masterpiece = filtered_masterpiece.sort_values(by='grade_score', ascending=False)
#     filtered_masterpiece = filtered_masterpiece.sort_values(by='wr', ascending=False)
    return filtered_masterpiece['masterpiece_seq'].head(10).values.tolist()

In [ ]:
era_based_popularity('르네상스')

[2, 3, 5, 4, 2, 1, 3, 3, 5, 4]

In [ ]:
user = pd.read_csv('https://raw.githubusercontent.com/hjy530/WOOYE/main/prototype/csv/user.csv', encoding='utf-8')
user_grade = pd.merge(grade, user, on="user_id")
user_grade.head(3)

,grade_seq,user_id,masterpiece_seq,grade_score,user_gender,user_age
0,1,이소이,2,6,0.0,17.0
1,2,이소이,1,8,0.0,17.0
2,3,이소이,14,2,0.0,17.0


In [ ]:
from numpy.ma.core import append
import collections

def user_top_era(user_id):
    bq2 = masterpiece_grade[['user_id', 'masterpiece_seq','masterpiece_era']]

    user_era = bq2['masterpiece_era'][masterpiece_grade['user_id'] == user_id]
    # print("User Era: ", user_era)
    
    top_era_indices = np.flip(user_era)
    counts = collections.Counter(top_era_indices)
    counts_w = sorted(counts.items(), key= lambda item: item[1], reverse=True)
    
    era_dict = dict(counts_w)

    era_list = list(era_dict.keys())

    return era_list[:3]

In [ ]:
user_top_era('이소이')

['바로크', '팝아트', '르네상스']

- Hybrid model

In [ ]:
knn_baseline = dump.load('KnnBaseline_model')
svd = dump.load('svd_model') 

In [ ]:
# List of users in testing data:
user_list = testdf['user_id'].unique()

In [ ]:
# type(testdf['userId'][0])
test_masterpiece = testdf[testdf['user_id'] == '이소이']
test_masterpiece.head()

,masterpiece_seq,masterpiece_title,masterpiece_artist,masterpiece_era,masterpiece_year,masterpiece_technique,masterpiece_material,grade_seq,user_id,grade_score
0,1,모나리자,레오나르도 다 빈치,르네상스,1503.0,인물화,유화,2,이소이,8
14,2,천지창조,미켈란젤로 부오나로티,르네상스,1512.0,종교화,벽화,1,이소이,6
37,4,아테네 학당,라파엘로 산치오,르네상스,1510.0,추상화,벽화,25,이소이,1
52,5,최후의 심판,미켈란젤로 부오나로티,르네상스,1541.0,종교화,벽화,26,이소이,5
65,6,십자가에서 내려지는 그리스도,피터 반 몰,바로크,1634.0,종교화,유화,9,이소이,7


In [ ]:
# Combined model predicion on testing data, using top movies to generate more movies based on movie similarity

def hybrid(user_id):
    user_masterpiece = testdf[testdf['user_id'] == user_id]
    user_masterpiece['est'] = user_masterpiece['masterpiece_seq'].apply(lambda x: 0.6*knnbaseline_algo.predict(user_id,x).est + 0.4*svd_algo.predict(user_id, x).est)    
    user_masterpiece = user_masterpiece.sort_values(by ='est', ascending=False).head(4)
    user_masterpiece['Model'] = 'SVD + CF'
#     user_masterpiece = user_masterpiece['masterpiece_seq'].values.tolist()
#     print("User liked masterpiece list: ", user_masterpiece)
    
    recommend_list = user_masterpiece[['masterpiece_seq', 'est', 'Model']]
    print(recommend_list.head())

#     top_masterpiece = user_masterpiece['masterpiece_seq'].iloc[0]
#     print("Top masterpiece id", top_masterpiece)
#     top_masterpiece_title = masterpiece['masterpiece_title'][masterpiece['masterpiece_seq'] == top_masterpiece].values[0]
#     print("Top masterpiece title", top_masterpiece_title)

    
    masterpiece_list = recommend_list['masterpiece_seq'].values.tolist()
    print(masterpiece_list)
    
    sim_masterpiece_list = []
    for masterpieces_id in masterpiece_list:
        # Call content based 
        masterpiece_title = masterpieces['masterpiece_title'][masterpieces['masterpiece_seq'] == masterpieces_id].values[0]
        sim_masterpiece = get_recommendations_new(masterpiece_title)
#       print(sim_masterpiece.values.tolist())
        sim_masterpiece_list.extend(sim_masterpiece)

    # Compute ratings for the popular movies
    for masterpieces_id in sim_masterpiece_list:
        pred_rating = 0.6*knnbaseline_algo.predict(user_id, masterpieces_id).est + 0.4*svdpp_algo.predict(user_id, masterpieces_id).est
        row_df = pd.DataFrame([[masterpieces_id, pred_rating, 'Movie similarity']], columns=['masterpiece_seq', 'est','Model'])
        recommend_list = pd.concat([recommend_list, row_df], ignore_index=True)
    
    # Popular based movies
    top_era_list = user_top_era(user_id)
    print("User top era list: ", top_era_list)
    
    popular_masterpiece = []
    for top_era in top_era_list:
        popular_masterpiece.extend(era_based_popularity(top_era))
    print("Final list: ", popular_masterpiece)
    
    # Compute ratings for the popular movies
    for masterpieces_id in popular_masterpiece:
        pred_rating = 0.6*knnbaseline_algo.predict(user_id, masterpieces_id).est + 0.4*svdpp_algo.predict(user_id, masterpieces_id).est
        row_df = pd.DataFrame([[masterpieces_id, pred_rating, 'Popularity']], columns=['masterpiece_seq', 'est','Model'])
        recommend_list = pd.concat([recommend_list, row_df], ignore_index=True)
    recommend_list = recommend_list.drop_duplicates(subset=['masterpiece_seq'])
    train_masterpieces_list = traindf[traindf['user_id']==user_id]['masterpiece_seq'].values.tolist()
    
    # Remove masterpieces in training for this user
    mask = recommend_list.masterpiece_seq.apply(lambda x: x not in train_masterpieces_list)
    recommend_list = recommend_list[mask]
    
    return recommend_list

In [ ]:
# traindf[traindf['user_id'] == '이소이'].sort_values(by = 'grade_score', ascending = False)
traindf[traindf['user_id'] == '이소이'].sort_values(by = 'grade_score', ascending = False)
# testdf[testdf['user_id'] == '이소이']

In [ ]:
masterpiece_ids = hybrid('도민준')

     masterpiece_seq  est     Model
18                 2  5.0  SVD + CF
311               26  5.0  SVD + CF
424               37  5.0  SVD + CF
415               36  5.0  SVD + CF
[2, 26, 37, 36]
User top era list:  ['표현주의', '인상주의', '사실주의']
Final list:  [39, 43, 43, 43, 39, 39, 43, 41, 43, 39, 35, 34, 37, 38, 37, 34, 34, 38, 34, 37, 33, 32, 31, 30, 31, 32, 29, 29, 33, 31]


In [ ]:
def get_title(x):
    mid = x['masterpiece_seq']
    return masterpiece['masterpiece_title'][masterpiece['masterpiece_seq'] == mid].values

In [ ]:
def get_era(x):
    mid = x['masterpiece_seq']
    return masterpiece['masterpiece_era'][masterpiece['masterpiece_seq'] == mid].values

In [ ]:
masterpiece_ids['masterpiece_title'] = masterpiece_ids.apply(get_title, axis=1)
masterpiece_ids['masterpiece_era'] = masterpiece_ids.apply(get_era, axis=1)

In [ ]:
masterpiece_ids.sort_values(by='est', ascending = False).head(10)

,masterpiece_seq,est,Model,masterpiece_title,masterpiece_era
5,3,5.000000,Movie similarity,[최후의 만찬],[르네상스]
8,6,5.000000,Movie similarity,[십자가에서 내려지는 그리스도],[바로크]
9,7,5.000000,Movie similarity,[야경],[바로크]
12,10,5.000000,Movie similarity,[와인잔이 있는 정물],[바로크]
16,27,5.000000,Movie similarity,[발코니 방],[낭만주의]
17,28,5.000000,Movie similarity,[금색 벨벳 천에 목련],[낭만주의]
70,29,5.000000,Popularity,[과일 정물],[사실주의]
18,1,4.807315,Movie similarity,[모나리자],[르네상스]
10,8,4.772300,Movie similarity,[진주귀걸이를 한 소녀],[바로크]
6,4,4.345912,Movie similarity,[아테네 학당],[르네상스]
